In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\JK\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [3]:
# Visit houseplantsexpert.com website
url = 'https://www.houseplantsexpert.com/a-z-list-of-house-plants.html'
browser.visit(url)
# time.sleep(5)


# Screen-scraping

In [4]:
html = browser.html
soup = bs(html, "html.parser")

#Scraping the webpage to pull the url for individual plants
plant_url = []

i = 0

for i in range(1,4):
    count = 0
    datas = soup.find('ul', class_="pagination"+str(i))
    for result in datas.find_all('a'):
        url = result.get('href')
        plant_url.append(url)
    
# plant_url


In [5]:
plant_info=[]

for i in range(0,len(plant_url)):
    try:
        browser.visit(plant_url[i])
        
        plant_dict = {}
        
        html = browser.html
        soup = bs(html, "html.parser")

        plant_data = soup.find_all('div', class_="container")

        #  scraping data and saving it to a dictionary
        plant_dict["origin"] = plant_data[3].find_all('td')[1].text
        plant_dict["Common Name"] = plant_data[3].find_all('td')[3].text.split('common)')[0].split('(')[0].strip()
        plant_dict["scientific_name"] = plant_data[3].find_all('td')[3].text.split('common)')[1].split('(')[0].strip()
        plant_dict["Max_Growth"] = plant_data[3].find_all('td')[5].text
        plant_dict["Poisonous_for_pets"] = plant_data[3].find_all('td')[7].text
        plant_dict["big_image_url"] = plant_data[3].find_all('img')[0]["src"]
        plant_dict["temperature"] = plant_data[4].find_all('td')[1].text

#         print(plant_dict["Common Name"])
        plant_info.append(plant_dict)
#             browser.back()

    except:
        print("error scraping "+ plant_dict["Common Name"])
        browser.back()
browser.quit()
# plant_info

error scraping Lycaste
error scraping Moth orchid, Phal
error scraping P. lily, Spathe flower
error scraping Poison or German primrose
error scraping Slipper Orchid
error scraping Heart leaf Philodendron, sweetheart plant
error scraping Peacock Plant, Cathedral Windows
error scraping Pin Stripe Calathea
error scraping Lithops pseudotruncatella
error scraping Lithops optica 'Rubra'
error scraping Mother Of Thousands, Devils backbone, Mexican Hat, Evil Genius
error scraping Watermelon Peperomia


# Datacleansing

In [6]:
# store the data into dataframe
df = pd.DataFrame(plant_info)
df.head()

,origin,Common Name,scientific_name,Max_Growth,Poisonous_for_pets,big_image_url,temperature
0,South America.,Amaryllis,. Hippeastrum,Height 30 in/75 cm.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,The amaryllis within the growing period (winte...
1,Eastern Africa.,African violet,. Saintpaulia,Diameter (micro - 3in) - (large - 16in).,Non toxic dogs and cats.,https://www.houseplantsexpert.com/assets/image...,Room temperatures of around 65°f / 16°c --- 7...
2,South America.,Angel wing begonia,. Begonia coccinea,4ft tall or more if not pruned.,Toxic for cats and dogs.,https://www.houseplantsexpert.com/assets/image...,Average room temperatures of 55 - 75°F (13 - 2...
3,South Africa.,"Barberton, Gerbera, Transvaal Daisy",. -- Gebera Jamesonii,24in tall.,"Non toxic for cats, dogs and horses.",https://www.houseplantsexpert.com/assets/image...,Average temperatures of 55 - 75°F (13 - 24°C) ...
4,South Mexico and Central America.,Spider Lily,. Hymenocallis littoralis,Height 2 - 3ft.,No.,https://www.houseplantsexpert.com/assets/image...,The Beach Spider Lily is capable of withstandi...


In [7]:
df.count()

origin                102
Common Name           102
scientific_name       102
Max_Growth            102
Poisonous_for_pets    102
big_image_url         102
temperature           102
dtype: int64

In [8]:
df.dtypes

origin                object
Common Name           object
scientific_name       object
Max_Growth            object
Poisonous_for_pets    object
big_image_url         object
temperature           object
dtype: object

In [9]:
df = df.dropna(how='any')
df.count()

origin                102
Common Name           102
scientific_name       102
Max_Growth            102
Poisonous_for_pets    102
big_image_url         102
temperature           102
dtype: int64

In [10]:
df["origin"] = df["origin"].str.replace('.','')

df["Common Name"]= df["Common Name"].str.strip()
df["scientific_name"]= df["scientific_name"].str.strip()

df["scientific_name"]= df["scientific_name"].str.replace('.','')
df["scientific_name"]= df["scientific_name"].str.replace('--','')
df["temperature"] = df["temperature"].str.replace('°','')
df.head()

C:\Users\JK\AppData\Local\Temp/ipykernel_3308/1581714428.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["origin"] = df["origin"].str.replace('.','')
C:\Users\JK\AppData\Local\Temp/ipykernel_3308/1581714428.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["scientific_name"]= df["scientific_name"].str.replace('.','')


,origin,Common Name,scientific_name,Max_Growth,Poisonous_for_pets,big_image_url,temperature
0,South America,Amaryllis,Hippeastrum,Height 30 in/75 cm.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,The amaryllis within the growing period (winte...
1,Eastern Africa,African violet,Saintpaulia,Diameter (micro - 3in) - (large - 16in).,Non toxic dogs and cats.,https://www.houseplantsexpert.com/assets/image...,Room temperatures of around 65f / 16c --- 75f...
2,South America,Angel wing begonia,Begonia coccinea,4ft tall or more if not pruned.,Toxic for cats and dogs.,https://www.houseplantsexpert.com/assets/image...,Average room temperatures of 55 - 75F (13 - 24...
3,South Africa,"Barberton, Gerbera, Transvaal Daisy",Gebera Jamesonii,24in tall.,"Non toxic for cats, dogs and horses.",https://www.houseplantsexpert.com/assets/image...,Average temperatures of 55 - 75F (13 - 24C) ar...
4,South Mexico and Central America,Spider Lily,Hymenocallis littoralis,Height 2 - 3ft.,No.,https://www.houseplantsexpert.com/assets/image...,The Beach Spider Lily is capable of withstandi...


# Screen-scraping - Etsy

In [11]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\JK\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [12]:
# Visit houseplantsexpert.com website
url = 'https://www.etsy.com/'
browser.visit(url)
# time.sleep(5)

In [13]:
plant_price_data = []

for i in range(0,len(df)):
# for i in range (0, 5):
    plant_price_list = []
    plant_price_dict ={}
    
    plant_price_dict["scientific_name"] = df["scientific_name"][i]
#     print(df["scientific_name"][i])
    #searching Etsy website to pull prices information

    search_url = url +"search?q=" + df["scientific_name"][i]
    browser.visit(search_url)
    time.sleep(1)

    html = browser.html
    soup = bs(html, "html.parser")

    # finding the price of the first 10 results
    for j in range(0,10):
        try:
#             print(i)
            search_results = soup.find_all('span', class_="currency-value")[j].text.strip()
            plant_price_list.append(search_results)
#             print(search_results)
            
            plant_price = [float(price) for price in plant_price_list]
#             print(plant_price)
            plant_price_dict["max_plant_price"] = max(plant_price)
            plant_price_dict["min_plant_price"] = min(plant_price)
            plant_price_dict["average_plant_price"] = round(sum(plant_price) /len(plant_price),2)

        except:
            search_results="Nan"
            plant_price = "Nan"
            plant_price_list="Nan"
            plant_price_dict["max_plant_price"] = "NaN"
            plant_price_dict["min_plant_price"] = "NaN"
            plant_price_dict["average_plant_price"] = "NaN"
            
        # converting string to float
        
        
#     print(average_plant_price)
    plant_price_data.append(plant_price_dict)           
browser.quit()

In [14]:
# plant_price_data

In [15]:
# store the data into dataframe
plant_price_df = pd.DataFrame(plant_price_data)
plant_price_df.head()

,scientific_name,max_plant_price,min_plant_price,average_plant_price
0,Hippeastrum,348.26,9.99,57.74
1,Saintpaulia,53.5,8.0,24.38
2,Begonia coccinea,30.0,4.99,15.09
3,Gebera Jamesonii,7.5,3.0,4.9
4,Hymenocallis littoralis,18.0,9.99,13.49


In [16]:
df = pd.merge(df, plant_price_df, on="scientific_name", how = "outer")
df.count()
df

,origin,Common Name,scientific_name,Max_Growth,Poisonous_for_pets,big_image_url,temperature,max_plant_price,min_plant_price,average_plant_price
0,South America,Amaryllis,Hippeastrum,Height 30 in/75 cm.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,The amaryllis within the growing period (winte...,348.26,9.99,57.74
1,Eastern Africa,African violet,Saintpaulia,Diameter (micro - 3in) - (large - 16in).,Non toxic dogs and cats.,https://www.houseplantsexpert.com/assets/image...,Room temperatures of around 65f / 16c --- 75f...,53.5,8.0,24.38
2,South America,Angel wing begonia,Begonia coccinea,4ft tall or more if not pruned.,Toxic for cats and dogs.,https://www.houseplantsexpert.com/assets/image...,Average room temperatures of 55 - 75F (13 - 24...,30.0,4.99,15.09
3,South Africa,"Barberton, Gerbera, Transvaal Daisy",Gebera Jamesonii,24in tall.,"Non toxic for cats, dogs and horses.",https://www.houseplantsexpert.com/assets/image...,Average temperatures of 55 - 75F (13 - 24C) ar...,7.5,3.0,4.9
4,South Mexico and Central America,Spider Lily,Hymenocallis littoralis,Height 2 - 3ft.,No.,https://www.houseplantsexpert.com/assets/image...,The Beach Spider Lily is capable of withstandi...,18.0,9.99,13.49
...,...,...,...,...,...,...,...,...,...,...
97,Mexico,Rat's Tail,Aporocactus Flagelliformis or Disocactus flag...,Stems can grow a few feet long.,Not known.,https://www.houseplantsexpert.com/image-files/...,Temperatures averaging 50F/10C - 75F/23.8C are...,NaN,NaN,NaN
98,South africa,Tiger Jaws,Faucaria Tigrina,Diameter 8in - 20cm approx. Leaves 6in - 15 long.,Not known.,https://www.houseplantsexpert.com/assets/image...,Temperatures averaging 70F/21C - 90F/32C are v...,14.99,3.58,6.55
99,South America,"Trailing jade, round leaf Peperomia",Peperomia rotundifolia,25 - 30 cm or longer trailing stems.,Non-toxic to cats and dogs.,https://www.houseplantsexpert.com/assets/image...,"Ideal temperatures of 65-75ºF (18-24ºC), and n...",19.99,7.99,12.06
100,Africa,Truncate living stone,Lithops pseudotruncatella,Height 1cm - 5cm max.,"Not known, but it's unlikely a pet will chew o...",\n https://www.houseplantsexpert.com...,The Lithop Pseudotruncatella is a plant that i...,14.99,2.54,7.2


In [17]:
countries_df = pd.read_csv("UNSD_country_to_regions.csv")
countries_df.head()

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Unnamed: 15
0,1,World,142.0,Asia,34.0,Southern Asia,NaN,NaN,Afghanistan,4,AF,AFG,x,x,NaN,NaN
1,1,World,150.0,Europe,154.0,Northern Europe,NaN,NaN,Åland Islands,248,AX,ALA,NaN,NaN,NaN,NaN
2,1,World,150.0,Europe,39.0,Southern Europe,NaN,NaN,Albania,8,AL,ALB,NaN,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN,NaN
4,1,World,9.0,Oceania,61.0,Polynesia,NaN,NaN,American Samoa,16,AS,ASM,NaN,NaN,x,NaN


In [18]:
countries_df.drop(columns=["Global Code", "Global Name", "Region Code", "Sub-region Code", "Intermediate Region Code",
                           "M49 Code", "ISO-alpha2 Code", "Least Developed Countries (LDC)", "Land Locked Developing Countries (LLDC)",
                           "Small Island Developing States (SIDS)", "Unnamed: 15"],axis=1,inplace=True)
countries_df.head()

,Region Name,Sub-region Name,Intermediate Region Name,Country or Area,ISO-alpha3 Code
0,Asia,Southern Asia,NaN,Afghanistan,AFG
1,Europe,Northern Europe,NaN,Åland Islands,ALA
2,Europe,Southern Europe,NaN,Albania,ALB
3,Africa,Northern Africa,NaN,Algeria,DZA
4,Oceania,Polynesia,NaN,American Samoa,ASM


In [19]:
df['origin'].unique() 

array(['South America', 'Eastern Africa', 'South Africa',
       'South Mexico and Central America', 'East Africa', 'East Asia',
       'Central and South America',
       'Middle East, Greek Islands and elsewhere', 'Brazil', 'Madagascar',
       'Central America, Mexico', 'Mexico and Central America',
       'South America ', 'China', 'Southeast Asia', 'Peru and Ecuador',
       'North Africa', 'South East Asia and East Australia', 'America',
       'China and Taiwan', 'Canary Islands', 'Japan & Taiwan',
       'Tropics and subtropics of Asia', 'Various pacific ocean areas',
       'Africa', 'Europe, Asia and Africa',
       'South India, Sri Lanka, Malaysia, Western Pacific Islands',
       'West Indies', 'Southern Europe', 'Western Africa',
       'Australia, Indonesia, China, Japan and India',
       'Southeast Asia and Western Pacific Ocean Areas',
       'Lord Howe Island (Australia - New South Wales State)',
       'South and Central America', 'Mexico', 'South-East-Asia', 'Fiji'

In [20]:
#straight replacements to match country df names
#East Africa to Eastern Africa
df.loc[df['origin'] == "East Africa", 'origin'] = "Eastern Africa"
#South Africa to Southern Africa
df.loc[df['origin'] == "South Africa", 'origin'] = "Southern Africa"
#South africa to Southern Africa
df.loc[df['origin'] == "South africa", 'origin'] = "Southern Africa"
#East Asia to Eastern Asia
df.loc[df['origin'] == "East Asia", 'origin'] = "Eastern Asia"
# Southeast Asia to South-eastern Asia
df.loc[df['origin'] == "Southeast Asia", 'origin'] = "South-eastern Asia"
# North Africa to Northern Africa
df.loc[df['origin'] == "North Africa", 'origin'] = "Northern Africa"
#America to United States of America
df.loc[df['origin'] == "America", 'origin'] = "United States of America"
#South East Asia and East Australia to South-eastern Asia, Australia
df.loc[df['origin'] == "South East Asia and East Australia", 'origin'] = "South-eastern Asia, Australia"
#Canary Islands to Western Sahara
df.loc[df['origin'] == "Canary Islands", 'origin'] = "Western Sahara"
#West Indies to Caribbean
df.loc[df['origin'] == "West Indies", 'origin'] = "Caribbean"
#Southeast Asia and Western Pacific Ocean Areas to South-eastern Asia, Oceania
df.loc[df['origin'] == "Southeast Asia and Western Pacific Ocean Areas", 'origin'] = "South-eastern Asia, Oceania"
#Lord Howe Island (Australia - New South Wales State) to Australia
df.loc[df['origin'] == "Lord Howe Island (Australia - New South Wales State)", 'origin'] = "Australia"
# South-East-Asia to South-eastern Asia
df.loc[df['origin'] == "South-East-Asia", 'origin'] = "South-eastern Asia"
#Madagascar and other Indian ocean islands to Madagascar
df.loc[df['origin'] == "Madagascar and other Indian ocean islands", 'origin'] = "Madagascar"
#Taiwan to China
df.loc[df['origin'] == "Taiwan", 'origin'] = "China"
# North and South Carolina to United States of America
df.loc[df['origin'] == "North and South Carolina", 'origin'] = "United States of America"
# Africa and then developed in Thailand to the Lactea crest to Africa
df.loc[df['origin'] == "Africa and then developed in Thailand to the Lactea crest", 'origin'] = "Africa"
#South Mexico to Mexico
df.loc[df['origin'] == "South Mexico", 'origin'] = "Mexico"
#North Mexico to Mexico
df.loc[df['origin'] == "North Mexico", 'origin'] = "Mexico"
#South East Asia and Australia to South-eastern Asia, Australia
df.loc[df['origin'] == "South East Asia and Australia", 'origin'] = "South-eastern Asia, Australia"
#Japan & Taiwan to Japan, China
df.loc[df['origin'] == "Japan & Taiwan", 'origin'] = "Japan, China"
#Tropics and subtropics of Asia to South-eastern Asia, Oceania
df.loc[df['origin'] == "Tropics and subtropics of Asia", 'origin'] = "South-eastern Asia, Oceania"
#South India, Sri Lanka, Malaysia, Western Pacific Islands to India, Sri Lanka, Malaysia, Oceania
df.loc[df['origin'] == "South India, Sri Lanka, Malaysia, Western Pacific Islands", 'origin'] = "India, Sri Lanka, Malaysia, Oceania"
#Various pacific ocean areas to Oceania
df.loc[df['origin'] == "Various pacific ocean areas", 'origin'] = "Oceania"
#South Mexico to Columbia = Mexico, Central America, Columbia
df.loc[df['origin'] == "South Mexico to Columbia", 'origin'] = "Mexico, Central America, Columbia"
#China and Taiwan to China
df.loc[df['origin'] == "China and Taiwan", 'origin'] = "China"
#Peru and Ecuador to Peru, Ecuador
df.loc[df['origin'] == "Peru and Ecuador", 'origin'] = "Peru, Ecuador"
#Australia, Indonesia, China, Japan and India to Australia, Indonesia, China, Japan, India 
df.loc[df['origin'] == "Australia, Indonesia, China, Japan and India", 'origin'] = "Australia, Indonesia, China, Japan, India"
#India, Nepal, Myanmar, China, Malaysia & others to India, Nepal, Myanmar, China, Malaysia
df.loc[df['origin'] == "India, Nepal, Myanmar, China, Malaysia & others", 'origin'] = "India, Nepal, Myanmar, China, Malaysia"
# Europe, Asia and Africa to Europe, Asia, Africa
df.loc[df['origin'] == "Europe, Asia and Africa", 'origin'] = "Europe, Asia, Africa"
#Middle East, Greek Islands and elsewhere to Western Asia, Greece 
df.loc[df['origin'] == "Middle East, Greek Islands and elsewhere", 'origin'] = "Western Asia, Greece"
#South Mexico and Central America = central america, mexico
df.loc[df['origin'] == "South Mexico and Central America", 'origin'] = "Central America, Mexico"
#Mexico and Central America
df.loc[df['origin'] == "Mexico and Central America", 'origin'] = "Central America, Mexico"
# South America '
df.loc[df['origin'] == "South America ", 'origin'] = "South America"
# South and Central America
df.loc[df['origin'] == "South and Central America", 'origin'] = "South America, Central America"
# Africa 
df.loc[df['origin'] == "Africa ", 'origin'] = "Africa"
#  Mexico
df.loc[df['origin'] == " Mexico", 'origin'] = "Mexico"
#Central and South America
df.loc[df['origin'] == "Central and South America", 'origin'] = "South America, Central America"

In [21]:
df['origin'].unique()

array(['South America', 'Eastern Africa', 'Southern Africa',
       'Central America, Mexico', 'Eastern Asia',
       'South America, Central America', 'Western Asia, Greece', 'Brazil',
       'Madagascar', 'China', 'South-eastern Asia', 'Peru, Ecuador',
       'Northern Africa', 'South-eastern Asia, Australia',
       'United States of America', 'Western Sahara', 'Japan, China',
       'South-eastern Asia, Oceania', 'Oceania', 'Africa',
       'Europe, Asia, Africa', 'India, Sri Lanka, Malaysia, Oceania',
       'Caribbean', 'Southern Europe', 'Western Africa',
       'Australia, Indonesia, China, Japan, India', 'Australia', 'Mexico',
       'Fiji', 'Brazil ', 'India, Nepal, Myanmar, China, Malaysia',
       'Asia', 'Mexico, Central America, Columbia'], dtype=object)

In [22]:
#add new column to df
df['origin_country_list'] = ''

In [23]:
#function to translate region names into country names
for i in df.index:
    to_add = ""
    current_origin = df['origin'][i]
    if ',' in current_origin: #multiple regional names
        new_list = current_origin.split(', ')
        for j in new_list: 
            for x in countries_df.index:
                if countries_df['Region Name'][x] == j or countries_df['Sub-region Name'][x] == j or countries_df['Intermediate Region Name'][x] == j or countries_df['Country or Area'][x] == j:
                    if to_add:
                        to_add = to_add + ", " + countries_df['Country or Area'][x]
                    else:
                        to_add = countries_df['Country or Area'][x]  
            df['origin_country_list'][i] = to_add #add array to location in df
    else:
        for x in countries_df.index:
            if countries_df['Region Name'][x] == current_origin or countries_df['Sub-region Name'][x] == current_origin or countries_df['Intermediate Region Name'][x] == current_origin or countries_df['Country or Area'][x] == current_origin:
                if to_add:
                    to_add = to_add + ", " + countries_df['Country or Area'][x]
                else:
                    to_add = countries_df['Country or Area'][x]  
        df['origin_country_list'][i] = to_add#add array to location in df


In [24]:
#check
df.head(20)

,origin,Common Name,scientific_name,Max_Growth,Poisonous_for_pets,big_image_url,temperature,max_plant_price,min_plant_price,average_plant_price,origin_country_list
0,South America,Amaryllis,Hippeastrum,Height 30 in/75 cm.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,The amaryllis within the growing period (winte...,348.26,9.99,57.74,"Argentina, Bolivia, Bouvet Island, Brazil, Chi..."
1,Eastern Africa,African violet,Saintpaulia,Diameter (micro - 3in) - (large - 16in).,Non toxic dogs and cats.,https://www.houseplantsexpert.com/assets/image...,Room temperatures of around 65f / 16c --- 75f...,53.5,8.0,24.38,"British Indian Ocean Territory, Burundi, Comor..."
2,South America,Angel wing begonia,Begonia coccinea,4ft tall or more if not pruned.,Toxic for cats and dogs.,https://www.houseplantsexpert.com/assets/image...,Average room temperatures of 55 - 75F (13 - 24...,30.0,4.99,15.09,"Argentina, Bolivia, Bouvet Island, Brazil, Chi..."
3,Southern Africa,"Barberton, Gerbera, Transvaal Daisy",Gebera Jamesonii,24in tall.,"Non toxic for cats, dogs and horses.",https://www.houseplantsexpert.com/assets/image...,Average temperatures of 55 - 75F (13 - 24C) ar...,7.5,3.0,4.9,"Botswana, Eswatini, Lesotho, Namibia, South Af..."
4,"Central America, Mexico",Spider Lily,Hymenocallis littoralis,Height 2 - 3ft.,No.,https://www.houseplantsexpert.com/assets/image...,The Beach Spider Lily is capable of withstandi...,18.0,9.99,13.49,"Belize, Costa Rica, El Salvador, Guatemala, Ho..."
5,Southern Africa,"Belladonna lily, March lily and naked lady.",Amaryllis belladonna,Height 24 in/60 cm.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,Temperatures from 65 - 75Ff (18 - 24c) are bes...,43.55,2.99,20.11,"Botswana, Eswatini, Lesotho, Namibia, South Af..."
6,Southern Africa,"Bird of Paradise, Strelitzia, Crane Flower",Strelitzia Reginae,Height 6ft (can be kept to 3ft indoors).,"Mildly toxic for cats, dogs",https://www.houseplantsexpert.com/assets/image...,The Bird of Paradise requires moderate tempera...,90.0,5.0,31.33,"Botswana, Eswatini, Lesotho, Namibia, South Af..."
7,South America,Blushing bromeliad,Neoregelia carolinae,Height 18 in/45 cm.,"Non toxic for cats, dogs",https://www.houseplantsexpert.com/image-files/...,Average to just above average temperatures of...,15.95,4.0,13.69,"Argentina, Bolivia, Bouvet Island, Brazil, Chi..."
8,Eastern Africa,Busy Lizzie and Impatient Lucy,Impatiens Walleriana,24in (60cm) in height.,Non-toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,Average room temperatures from 65F/18.3C - 75F...,12.0,3.0,5.89,"British Indian Ocean Territory, Burundi, Comor..."
9,Southern Africa,Calla lily,Zantedeschia aethiopica,Flower stalk 3ft (90cm) in height.,Toxic to cats and dogs.,https://www.houseplantsexpert.com/image-files/...,These flowers can be rather picky about their ...,20.92,1.42,9.02,"Botswana, Eswatini, Lesotho, Namibia, South Af..."


In [25]:
# exporting to csv file
df.to_csv("plant_list.csv", index=True, header = True)

In [26]:
#check
df['origin_country_list'][11]

'Argentina, Bolivia, Bouvet Island, Brazil, Chile, Colombia, Ecuador, Falkland Islands (Malvinas), French Guiana, Guyana, Paraguay, Peru, South Georgia and the South Sandwich Islands, Suriname, Uruguay, Venezuela, Belize, Costa Rica, El Salvador, Guatemala, Honduras, Mexico, Nicaragua, Panama'

# export data to SQLite database

In [27]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [28]:
Base = declarative_base()

In [29]:
class Plant(Base):
    __tablename__ = 'houseplants'
    id = Column(Integer, primary_key=True)
    common_name = Column(String(100))
    scientific_name = Column(String(100))
    max_growth = Column(String(100))
    poisonous = Column(String(100))
    photo_url = Column(String(255))
    temperature = Column(String(100))
    max_plant_price = Column(String(100))
    min_plant_price = Column(String(100))
    avg_plant_price = Column(String(100))
    origin_countries = Column(String(255))

In [30]:
#Create db connection
engine = create_engine("sqlite:///houseplants.sqlite")
conn = engine.connect()

In [31]:
#creates tables from python classes
Base.metadata.create_all(engine)

In [32]:
#create connection to edit db
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [33]:
#for loop this with scraped data to add to db
for row in df.itertuples():
    plant_toadd = Plant(common_name=row[2], scientific_name=row[3], max_growth=row[4], poisonous=row[5], photo_url=row[6], temperature=row[7], max_plant_price=row[8], min_plant_price=row[9], avg_plant_price=row[10], origin_countries=row[11])
    session.add(plant_toadd)

In [34]:
#push additions to db
session.commit()

In [35]:
#close session
session.close()

In [36]:
session.query(Plant.origin_countries).all()

[('Argentina, Bolivia (Plurinational State of), Bouvet Island, Brazil, Chile, Colombia, Ecuador, Falkland Islands (Malvinas), French Guiana, Guyana, Paraguay, Peru, South Georgia and the South Sandwich Islands, Suriname, Uruguay, Venezuela (Bolivarian Republic of)',),
 ('British Indian Ocean Territory, Burundi, Comoros, Djibouti, Eritrea, Ethiopia, French Southern Territories, Kenya, Madagascar, Malawi, Mauritius, Mayotte, Mozambique, Réunion, Rwanda, Seychelles, Somalia, South Sudan, Uganda, United Republic of Tanzania, Zambia, Zimbabwe',),
 ('Argentina, Bolivia (Plurinational State of), Bouvet Island, Brazil, Chile, Colombia, Ecuador, Falkland Islands (Malvinas), French Guiana, Guyana, Paraguay, Peru, South Georgia and the South Sandwich Islands, Suriname, Uruguay, Venezuela (Bolivarian Republic of)',),
 ('Botswana, Eswatini, Lesotho, Namibia, South Africa',),
 ('Belize, Costa Rica, El Salvador, Guatemala, Honduras, Mexico, Nicaragua, Panama, Mexico',),
 ('Botswana, Eswatini, Lesotho